# Drift Model Testing

__Intended Process:__

- STEP 1: get stuff out of a data file

- UDD format data --> generate drift times (t1 - t0) then input into the equation to get drift radii (and uncertainties)
- use same data and get simulated radii from existing falaise
- plot radii versus each other to see how the models compare
- is there "truth data" for simulated data? (Ask Cheryl) -- see quantitatively how well Betsy's drift model works to estimate radius, get a quantitative result for how well it works compared to existing falaise

Next steps:  
- use real data as opposed to simulated
- quantify impact of implementing more detailed electric field
- track fitting?

Barriers to progress: 
- .brio format? Look at how Betsy got her data
    - used .root filed (can use Cheryl's code to convert)
- which pressure are we using again?

In [18]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time as tm

In [2]:
data = uproot.open('snemo_run-840_udd.root')

In [3]:
data['SimData;1'].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
header.runnumber     | int32_t                  | AsDtype('>i4')
header.eventnumber   | int32_t                  | AsDtype('>i4')
header.date          | int32_t                  | AsDtype('>i4')
header.runtype       | int32_t                  | AsDtype('>i4')
header.simulated     | bool                     | AsDtype('bool')
header.real          | bool                     | AsDtype('bool')
tracker.nohits       | int32_t                  | AsDtype('>i4')
tracker.id           | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
tracker.module       | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
tracker.side         | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
tracker.layer        | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
tracker.column       | std::vector<int32_t>     | AsJagge

In [20]:
data['SimData;1']['digitracker.anodetimestampR0'].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
digitracker.anodetim | std::vector<std::vector< | AsObjects(AsVector(True, AsVec


In [9]:
t0 = data['SimData;1']['digitracker.anodetimestampR0'].array(library='np')
t1 = data['SimData;1']['digitracker.anodetimestampR1'].array(library='pd')

In [62]:
n = 2
print(len(t0[n]))
print(len(t1[n]))

10
10


In [76]:
len(t0)

9590

In [109]:
t0[0][1]

<STLVector [847289127] at 0x026ba5d42a08>

In [120]:
test_vector_0 = np.array(data['SimData;1']['digitracker.anodetimestampR0'].array()[5])
test_vector_1 = np.array(data['SimData;1']['digitracker.anodetimestampR1'].array()[5])

In [33]:
np.array(data['SimData;1']['digitracker.anodetimestampR0'].array(library='np')[0])

array([[847289447],
       [847289127],
       [847289197],
       [847289122],
       [847289281],
       [847289437],
       [847289134],
       [847289144],
       [847289316],
       [847286043],
       [847285664]], dtype=int64)

In [ ]:
ti = tm.time()
maxes = []
exceptions = []
count = 0

for n in np.arange(len(t0)):
    times = []
    try:
        test_vector_0 = np.array(data['SimData;1']['digitracker.anodetimestampR0'].array(library='np', dtype='object')[n])
        test_vector_1 = np.array(data['SimData;1']['digitracker.anodetimestampR1'].array(library='np', dtype='object')[n])

        for i in np.arange(len(test_vector_0)):
            time = test_vector_1[i] - test_vector_0[i]
            if time > 0:
                if time < 10000:
                    times.append(time)
        maxes.append(max(times))
        
    except: 
        exceptions.append(count)
    
    count += 1
    
        
print(max(maxes))
tf = tm.time()
print('Runtime:', tf - ti)

In [20]:
len(exceptions)

218

In [31]:
layers = data['SimData;1']['digitracker.layer'].array(library='np')

In [32]:
layers[3]

array([8, 7, 2, 0, 1, 4, 5, 3, 3, 1, 5])

In [8]:
# dictionary for a, b values
# each cell type has 9 values -- 3 sets of a, b pairs corresponding to 3 pressures (850, 880, 910) and the tx value for 
# each pressure in the third entry of each tuple

# 'entry' : [(a_850, b_850, tx_850), (a_880, b_880, tx_880), (a_910, b_910, tx_910)]
ab_vals = {'center_in': [(8.28, -0.9, 2.95), (8.53, -0.9, 2.97), (8.77, -0.9, 3.07)],
           'center_out': [(3.86, -1.99, 2.95), (4.19, -1.93, 2.97), (4.55, -1.9, 3.06)]
          }

In [11]:
# r(t) = (t/a)**(1/(1-b))

def define_io(t_drift, tx):
    """ Defines whether the particle is in the inner or outer section of the cell.
    param t_drift: The measured drift time (t1 - t0)
    type t_drift: float
    param tx: The reference value of t
    type t_drift: float
    """
    if t_drift > tx:
        inner = False
    else:
        inner = True
    return inner

def find_params(inner, region, pressure):
    """ Consults the ab_vals dictionary to determine a and b parameter values.
    param inner: Defines where the particle is within the cell
    type inner: bool
    param region: Defines the cell region as 'center', 'edge', or 'corner'
    type region: str
    param pressure: Defines the pressure within the tracker (850, 880, or 910)
    type pressure: float
    """
    cell_type = region
    
    if inner is True: 
        cell_type.append('_in')
    else:
        cell_type.append('_out')
        
    if pressure is 850:
        n = 0
    if pressure is 880:
        n = 1
    if pressure is 910:
        n = 2 
        
    params = ab_vals(cell_type[n])
    a = params[0]
    b = params[1]
    tx = params[2]
    
    return params

# since defining the above function, can remove cell_type and pressure from calc_radius and just reference find_params instead

def calc_radius(t_drift, cell_type='center_in', pressure=880):
    """ Calculated the radius of the particle based on drift time and cell conditions.
    param t_drift: The measured drift time (t1 - t0)
    type t_drift: float
    param cell_type: indicates what type of cell the particle is in, references an entry in the ab_vals dictionary
    type cell_type: str, kwarg
    param pressure: indicates the pressure in the tracking chamber, determines which tuple in the cell type dictionary entry should be used 
                    to define a and b
    type pressure: float, kwarg
    """
    if pressure is 850:
        n = 0
    if pressure is 880:
        n = 1
    if pressure is 910:
        n = 2
    
    params = ab_vals(cell_type[n])
    a = params[0]
    b = params[1]
    
    rad = (t_drift / a)**(1 / (1 + 0.9))
    return rad

def calc_uncertainty():